# PAM Algorithmus
Der PAM-Algorithmus (Partitioning Around Medoids) ist ein Clustering-Algorithmus, der iterativ Medoide auswählt, um Cluster zu bilden.

In [ ]:
import random
import math

import plotly.express as px

from tui_dsmt.clustering import PAM
from tui_dsmt.clustering.datasets import clustering_example4

## Inhaltsverzeichnis
- [Voraussetzungen](#Voraussetzungen)
  - [Definition: Medoid](#Definition-Medoid)
  - [Abstandsmaß](#Abstandsmaß)
  - [Kompaktheit](#Kompaktheit)
- [Algorithmus](#Algorithmus)
- [Implementierung in scikit-learn](#Implementierung-in-scikit-learn)
- [Zusammenfassung](#Zusammenfassung)

## Voraussetzungen
### Definition: Medoid
Ein Medoid ist ein Punkt, der einen Cluster repräsentiert. Im Gegensatz zum Zentroiden ist der Medoid kein künstlicher Punkt, sondern tatsächlich ein Objekt des Datensatzes.

### Abstandsmaß
Als Abstandsmaß kommt in diesem Notebook erneut der euklidische Abstand zum Einsatz:

In [ ]:
def dist(o1, o2):
    return math.sqrt(sum((c1 - c2) ** 2 for c1, c2 in zip(o1, o2)))

### Kompaktheit
Die Kompaktheit für einen spezifischen Cluster $C$ definiert sich wie folgt: $$ TD(C) = \sum_{p \in C} dist(p, m_c) $$

Die Kompaktheit für das gesamte Clustering wird dagegen als Summe der Kompaktheit aller Cluster ausgedrückt: $$ TD = \sum_{i=1}^{k} TD(C_i) $$

Wir verwenden an dieser Stelle die Funktion `pam_td`. Sie ordnet jedes Objekt dem nächstgelegenen Medoiden zu und berechnet parallel die Kompaktheit für das Clustering.

In [ ]:
def pam_td(dataset, medoids):
    clusters = [
        min(
            (i for i in range(len(medoids))),
            key=lambda i: dist(o, medoids[i])
        )
        for o in dataset
    ]

    td = sum(dist(o, medoids[i]) for o, i in zip(dataset, clusters))

    return clusters, td

## Algorithmus
Beim PAM-Algorithmus handelt es sich um einen iterativen Algorithmus, der schrittweise versucht die Rolle eines Medoiden einem anderen Punkt des Datensatzes zuzuweisen, um die Kompaktheit des Clusterings zu senken.

Zuerst werden nun initiale Medoiden zufällig bestimmt, z.B. durch Sampling aus dem gesamten Datensatz. Ihre Anzahl entspricht dabei der Anzahl der gewünschten Cluster. In jedem Durchlauf werden nun die folgenden Schritte wiederholt, bis keine Verringerung der Kompaktheit mehr möglich ist:
- Für alle Paare bestehend aus einem Medoiden $M$ und einem Nicht-Medoiden $N$ wird die zu erwartende Änderung der Kompaktheit berechnet.
- Es wird das Paar mit der maximalen Reduktion der Kompaktheit ausgewählt.
- Verbessert die maximale Reduktion das Ergebnis, d.h. die Kompaktheit sinkt, wird der entsprechende Medoid mit dem Nicht Medoiden vertauscht und mit der nächsten Iteration fortgefahren.

In [ ]:
def pam(dataset, k):
    # Die Medoide werden zufällig aus dem Datensatz ausgewählt.
    medoids = random.sample(dataset, k=3)

    # Solange eine Verringerung von TD im letzten Durchlauf
    # erreicht wurde ...
    delta_TD = -math.inf
    while delta_TD < 0:
        # Zur Berechnung der Veränderung der Kompaktheit muss
        # zuerst die Kompaktheit bestimmt werden.
        _, TD = pam_td(dataset, medoids)

        # Jedes Paar aus Medoid und Nicht-Medoid soll
        # betrachtet werden, um den vielversprechendsten
        # Tausch bezüglich der Verringerung der
        # Kompaktheit zu finden.
        best_pair, delta_TD = None, math.inf

        for i, M in enumerate(medoids):
            for N in dataset:
                if M == N:
                    continue

                # Vertauschen
                medoids[i] = N

                # neuen TD-Wert unter Annahme des Tauschs
                # berechnen
                _, TD_MN = pam_td(dataset, medoids)
                if delta_TD > TD_MN - TD:
                    best_pair = (i, N)
                    delta_TD = TD_MN - TD

                # Zurücktauschen
                medoids[i] = M

        # Tausch des besten Paares durchführen
        if delta_TD < 0:
            i, N = best_pair
            medoids[i] = N

    return medoids

Das Ergebnis lässt sich am Beispieldatensatz veranschaulichen. Bitte beachten Sie, dass die x- und y-Achsen möglicherweise **nicht** gleich skaliert sind!

In [ ]:
dataset = list(zip(clustering_example4['x'], clustering_example4['y']))
medoids = pam(dataset, 3)
clusters, _ = pam_td(dataset, medoids)

px.scatter(clustering_example4, x='x', y='y',
           color=['M' if dataset[i] in medoids else str(c) for i, c in enumerate(clusters)])

Auch diesen Algorithmus können Sie schrittweise bei der Arbeit verfolgen:

In [ ]:
PAM(dataset, k=3)

## Implementierung in scikit-learn
In `scikit-learn` direkt existiert keine Implementierung des PAM-Algorithmus. Sie ist dagegen in der Erweiterung `scikit-learn-extra` enthalten und wird analog zu k-Means angewendet:

In [ ]:
from sklearn_extra.cluster import KMedoids

KMedoids(3, method='pam').fit_predict(clustering_example4[['x', 'y']])

## Zusammenfassung
Der PAM-Algorithmus findet Medoide, also Stellvertreter eines Clusters, die tatsächlich Teil des Datensatzes sind. Durch die häufigen Vertauschungsoperationen mit anschließender Prüfung der Kompaktheit besitzt der Algorithmus jedoch eine hohe Laufzeit.